# Advection diffusion

This section shows how to solve
$$
-\nabla(\varepsilon \nabla u)+\beta\cdot\nabla u = f \text{ in } \Omega
$$
$$
u=0 \text{ on } \partial\Omega
$$

### Finite element spaces with Dirichlet conditions

In [ ]:
from ngsolve import *
from netgen.occ import *
from ngsolve.webgui import Draw

In [ ]:
maxh = 0.05
mesh = Mesh(unit_square.GenerateMesh(maxh=maxh))

In [ ]:
fes = H1(mesh, order=2, dirichlet="bottom|top|left|right") #need higher order to see effects between GLS and SUPG
u = fes.TrialFunction()
v = fes.TestFunction()

In [ ]:
eps = 0.1
beta = (50,0)
cbeta = CoefficientFunction(beta)
f = 10

### Weak Formulation
$$ \varepsilon\int\nabla u\cdot\nabla v + \int \beta\cdot\nabla u v = \int fv$$

In [ ]:
a = BilinearForm(fes)
a += (eps*grad(u)*grad(v)+cbeta*grad(u)*v) * dx
a.Assemble()

l = LinearForm(fes)
l += f*v*dx
l.Assemble()

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * l.vec 
Draw(gfu)

### Strongly Consistent Stabilization
approximate $h_K$ with $\max_{K} h_K$

##### SUPG $\rho=0$
add $$+\sum_K \delta_K\int_K (-\varepsilon\Delta u + \beta\cdot\nabla u - f)(\beta\cdot\nabla v)\frac{h_k}{|\beta|}$$

In [ ]:
delta = 0.3
lapu = Trace(u.Operator("hesse"))
lapv = Trace(v.Operator("hesse"))


a = BilinearForm(fes)
a += ( eps*grad(u)*grad(v) + cbeta*grad(u)*v )*dx
a += delta * (-eps*lapu + cbeta*grad(u)) * (cbeta*grad(v)) * maxh/Norm(cbeta)*dx
a.Assemble()

l = LinearForm(fes)
l += f*v*dx
l += delta* f * (cbeta*grad(v)) * maxh/Norm(cbeta)*dx
l.Assemble()

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * l.vec 
Draw(gfu)

#### GLS $\rho = 1$

In [ ]:
delta = .3
lapu = Trace(u.Operator("hesse"))
lapv = Trace(v.Operator("hesse"))

a = BilinearForm(fes)
a += (eps*grad(u)*grad(v)+cbeta*grad(u)*v) * dx
a += delta * (-eps*lapu + cbeta*grad(u)) * (-eps*lapv + cbeta*grad(v)) * maxh/Norm(cbeta)*dx
a.Assemble()

l = LinearForm(fes)
l += f*v * dx
l += delta * f * (-eps*lapv + cbeta*grad(v)) * maxh/Norm(cbeta)*dx
l.Assemble()

In [ ]:
gfu = GridFunction(fes)
gfu.vec.data = a.mat.Inverse(freedofs=fes.FreeDofs()) * l.vec 
Draw(gfu)

In [ ]:
from netgen.meshing import MeshingParameters
mp = MeshingParameters (maxh = maxh)
minh = 0.01
for j in range(0, int(1/minh)+1):
    xk = 1.0
    yk = j*minh
    mp.RestrictH (x=xk, y=yk, z=0, h=minh )
mesh = Mesh(unit_square.GenerateMesh(mp=mp))
Draw(mesh)